This is an experiment to check if we can clone a voice for the VITS tts. Here we will use tts models from MMS. 

In [28]:
import os

In [29]:
# create filelist from lj speech
data_dir = 'data/LJSpeech-1.1/'
with open(data_dir+'metadata.csv', 'r') as file:
    lines = file.readlines()
print(len(lines))
print(lines[0])

13100
LJ001-0001|Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition|Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition



In [30]:
# preprocess lines
# filter short and long senteces
lines_filter_words = []
min_words, max_words = 2, 10
for line in lines:
    wav_path, text, _ = line.split('|')
    no_words = len(text.split())
    if no_words > min_words and no_words < max_words:
        # add .wav and data_dir
        wav_path = data_dir + 'wavs/' + wav_path + '.wav'
        line = '|'.join([wav_path, text]) + '\n'
        lines_filter_words.append(line)
print(len(lines_filter_words))
lines_filter_words[0]

1644


'data/LJSpeech-1.1/wavs/LJ001-0002.wav|in being comparatively modern.\n'

In [31]:
# take few minutes of data
import sox

target_duration = 20 # min
lines_filter_dur = []

total_duration = 0.0
for line in lines_filter_words:
    wav_path = line.split('|')[0]
    total_duration += sox.file_info.duration(wav_path)
    lines_filter_dur.append(line)
    if total_duration > target_duration * 60:
        break
print(total_duration/60)
print(len(lines_filter_dur))

20.060153566666678
395


In [32]:
# this is for google colab as I dont have large gpus
# copy filter wav files to a separte folder
import shutil
if os.path.exists('data/ljspeech_filter/'):
     shutil.rmtree('data/ljspeech_filter/')
os.makedirs('data/ljspeech_filter/', exist_ok=True)
os.makedirs('data/ljspeech_filter/wavs', exist_ok=True)

for line in lines_filter_dur:
    source_path = line.split('|')[0]
    shutil.copy(source_path, source_path.replace('LJSpeech-1.1','ljspeech_filter'))


In [34]:
# save filelist to VITS dir
file = open('filelists/ljspeech_filelist.txt', "w", encoding='utf8')
for line in lines_filter_dur:
    line = line.replace('LJSpeech-1.1','ljspeech_filter')
    file.write(line)
file.close()

In [35]:
# preprocess filelist for VITS
os.system('python preprocess.py --text_index 1 --filelists filelists/ljspeech_filelist.txt')

START: filelists/ljspeech_filelist.txt


0

Create configs/finetune.json by 

- copying the configs/ljs_base.json
- changing the training file path
- change sampling rate 22050 to 16000
- learning_rate decreased to 2e-5

In [ ]:
os.system('python finetune.py -c configs/finetune.json -m finetuned_ljspeech')


In [ ]:
# download data
!wget https://dl.fbaipublicfiles.com/mms/tts/eng.tar.gz 
!tar -xvzf eng.tar.gz
